# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [72]:
# Dependencies and Setup
from functools import reduce
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [73]:
# Calculate the total number of unique schools
school_count = len(school_data_complete['school_name'].unique())
school_count

15

In [74]:
# Calculate the total number of students
student_count = len(school_data_complete['Student ID'])
student_count

39170

In [75]:
# Calculate the total budget
total_budget = school_data_complete['budget'].unique().sum()
total_budget

24649428

In [76]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()
average_math_score

78.98537145774827

In [77]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()
average_reading_score

81.87784018381414

In [78]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [79]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

In [80]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [81]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame(
    {"Total School" : school_count,
     "Total Students" : student_count,
     "Total Budget" : total_budget,
     "Average Math Score" : average_math_score,
     "Average Reading Score" : average_reading_score,
     "% Passing Math" : passing_math_percentage,
     "% Passing Reading" : passing_reading_percentage,
     "% Overall Passing" : overall_passing_rate
    
    },
    index = [0]
)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total School,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [82]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [83]:
# Calculate the total student count
per_school_counts = school_data_complete['school_name'].value_counts()
per_school_counts

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

In [84]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3362352537.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]


In [85]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(['school_name']).mean()['math_score']
per_school_reading = school_data_complete.groupby(['school_name']).mean()['reading_score']
per_school_math = per_school_math.reset_index().rename(columns= {'math_score' : 'Average Math Score'})
per_school_reading = per_school_reading.reset_index().rename(columns= {'reading_score' : 'Average Reading Score'})
math_reading_df = pd.merge(per_school_math, per_school_reading, how='left', on='school_name')
math_reading_df = math_reading_df.rename(columns={'school_name' : ' '})

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3229222901.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_math = school_data_complete.groupby(['school_name']).mean()['math_score']
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3229222901.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_reading = school_data_complete.groupby(['school_name']).mean()['reading_score']


In [86]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]

In [87]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]


In [88]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

In [89]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

passing_math_df = per_school_passing_math.reset_index().rename(columns={'index' : ' ', 0 : '% Passing Math'})
passing_reading_df = per_school_passing_reading.reset_index().rename(columns={'index' : ' ', 0 : '% Passing Reading'})
overall_passing_df = overall_passing_rate.reset_index().rename(columns={'index' : ' ', 0 : '% Overall Passing'})
passing_df = pd.merge(pd.merge(passing_math_df, passing_reading_df, on=' '), overall_passing_df, on=' ')

In [90]:
counts_df = per_school_counts.reset_index().rename(columns={'school_name' : 'Total Students', 'index': ' '})
type_df = school_types.reset_index().rename(columns={'school_name' : ' ', 'type' : 'School Type'})
budget_df = per_school_budget.reset_index().rename(columns={'school_name' : ' ', 'budget' : 'Total School Budget'})
capita_df = per_school_capita.reset_index().rename(columns={'index' : ' ', 0 : 'Per Student Budget'})
counts_type_df = pd.merge(counts_df, type_df, how='left', on=' ')
counts_type_budget_df = pd.merge(counts_type_df, budget_df, how='left', on=' ')
counts_type_budget_capita_df = pd.merge(counts_type_budget_df, capita_df, how='left', on=' ')
final_df = pd.merge(counts_type_budget_capita_df, math_reading_df, how='left', on=' ')
final_df = pd.merge(final_df, passing_df, how='left', on=' ')

In [91]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = final_df[[' ', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
1,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
2,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
3,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
4,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
5,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
6,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
7,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
8,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
9,Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333


## Highest-Performing Schools (by % Overall Passing)

In [92]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values('% Overall Passing', ascending=False)
top_schools.head(5)

,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
8,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
11,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
12,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
7,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
13,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [93]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values('% Overall Passing', ascending=True)
bottom_schools.head(5)

,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
3,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
4,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
5,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
2,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
1,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [94]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(by='school_name').mean()
tenth_graders_scores = tenth_graders.groupby(by='school_name').mean()
eleventh_graders_scores = eleventh_graders.groupby(by='school_name').mean()
twelfth_graders_scores = twelfth_graders.groupby(by='school_name').mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"].reset_index().rename(columns={'school_name':' ', 'math_score':'9th'})
tenth_grader_math_scores = tenth_graders_scores["math_score"].reset_index().rename(columns={'school_name':' ', 'math_score':'10th'})
eleventh_grader_math_scores = eleventh_graders_scores["math_score"].reset_index().rename(columns={'school_name':' ', 'math_score':'11th'})
twelfth_grader_math_scores = twelfth_graders_scores["math_score"].reset_index().rename(columns={'school_name':' ', 'math_score':'12th'})

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.merge(pd.merge(pd.merge(ninth_grade_math_scores, tenth_grader_math_scores, on=' '), eleventh_grader_math_scores, on=' '), twelfth_grader_math_scores, on=' ')

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3424871844.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ninth_graders_scores = ninth_graders.groupby(by='school_name').mean()
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3424871844.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tenth_graders_scores = tenth_graders.groupby(by='school_name').mean()
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3424871844.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify n

,,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [95]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(by='school_name').mean()
tenth_graders_scores = tenth_graders.groupby(by='school_name').mean()
eleventh_graders_scores = eleventh_graders.groupby(by='school_name').mean()
twelfth_graders_scores = twelfth_graders.groupby(by='school_name').mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"].reset_index().rename(columns={'school_name':' ', 'reading_score':'9th'})
tenth_grader_reading_scores = tenth_graders_scores["reading_score"].reset_index().rename(columns={'school_name':' ', 'reading_score':'10th'})
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"].reset_index().rename(columns={'school_name':' ', 'reading_score':'11th'})
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"].reset_index().rename(columns={'school_name':' ', 'reading_score':'12th'})

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.merge(pd.merge(pd.merge(ninth_grade_reading_scores, tenth_grader_reading_scores, on=' '), eleventh_grader_reading_scores, on=' '), twelfth_grader_reading_scores, on=' ')

# Minor data wrangling
#reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\1840412266.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ninth_graders_scores = ninth_graders.groupby(by='school_name').mean()
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\1840412266.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tenth_graders_scores = tenth_graders.groupby(by='school_name').mean()
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\1840412266.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify n

,,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [96]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [97]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [98]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(capita_df["Per Student Budget"], spending_bins, labels=spending_labels)
school_spending_df

,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
1,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,<$585
2,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$630-645
3,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247,$630-645
4,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$585-630
5,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
6,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,<$585
7,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567,$645-680
8,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,$645-680
9,Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333,$585-630


In [99]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\4149390191.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\4149390191.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\4149390191.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.

In [100]:
# Assemble into DataFrame
spending_summary = pd.merge(pd.merge(pd.merge(pd.merge(spending_math_scores, spending_reading_scores, on='Spending Ranges (Per Student)'), spending_passing_math, on='Spending Ranges (Per Student)'), spending_passing_reading, on='Spending Ranges (Per Student)'), overall_passing_spending, on='Spending Ranges (Per Student)')

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,80.454563,82.393085,80.366901,88.180080,71.899192
$585-630,80.215193,82.498978,79.818510,89.148123,72.282026
$630-645,80.210854,82.305395,80.094762,88.519162,71.751829
$645-680,80.988503,83.049330,84.561706,91.631963,78.477073


## Scores by School Size

In [101]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [102]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=size_labels)
per_school_summary

,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
1,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
2,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
3,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247,Large (2000-5000)
4,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
5,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
6,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
7,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567,Large (2000-5000)
8,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
9,Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333,Medium (1000-2000)


In [103]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]
size_dfs = [size_math_scores, size_reading_scores, size_passing_math, size_passing_reading, size_overall_passing]

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\75997360.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\75997360.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\75997360.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_o

In [104]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = reduce(lambda  left,right: pd.merge(left,right,on=['School Size'], how='outer'), size_dfs)

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [105]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()["% Overall Passing"]

# Use the code provided to select new column data
#average_math_score_by_type = type_math_scores["Average Math Score"]
#average_reading_score_by_type = type_reading_scores["Average Reading Score"]
#average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
#average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
#average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

type_dfs = [type_math_scores, type_reading_scores, type_passing_math, type_passing_reading, type_overall_passing]

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3440894562.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3440894562.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_21344\3440894562.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, num

In [106]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = reduce(lambda  left,right: pd.merge(left,right,on=['School Type'], how='outer'), type_dfs)

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
